Hello World!
This notebook describes the decision tree based Machine Learning model I have created
to segment the users of Habits app.

# Looking around the data set and getting all colums in required format

In [43]:
# Import the required modules
import pandas as pd
import numpy as np

In [27]:
# simple function to read in the user data file.
# the argument parse_dates takes in a list of colums, which are to be parsed as date format
user_data_raw = pd.read_csv("janacare_user-engagement_Aug2014-Apr2016.csv", parse_dates = [-3,-2,-1])

In [28]:
# data metrics
user_data_raw.shape # Rows , colums

(372, 19)

In [29]:
# data metrics
user_data_raw.dtypes # data type of colums

user_id                                                        float64
num_modules_consumed                                           float64
num_glucose_tracked                                            float64
num_of_days_steps_tracked                                      float64
num_of_days_food_tracked                                       float64
num_of_days_weight_tracked                                     float64
insulin_a1c_count                                              float64
cholesterol_count                                              float64
hemoglobin_count                                               float64
watching_videos (binary - 1 for yes, blank/0 for no)           float64
weight                                                         float64
height                                                           int64
bmi                                                              int64
age                                                              int64
gender

As is visible from the last column data type, Pandas is not recognising it as date type format. 
This will make things difficult, so I delete this particular column and add a new one.
Since the data in *age_on_platform* can be recreated from *last_activity* & *first_login* colums

In [30]:
# drop last column
user_data_del_last_col = user_data_raw.drop("age_on_platform", 1)

In [31]:
# Check if colums has been deleted. Number of column changed from 19 to 18
user_data_del_last_col.shape

(372, 18)

In [32]:
# Copy data frame 'user_data_del_last_col' into a new one
user_data = user_data_del_last_col

In [33]:
# Create new column 'age_on_platform' which has the corresponding value in date type format
user_data["age_on_platform"] = user_data_del_last_col["last_activity"]-user_data_del_last_col["first_login"]

In [34]:
# Check the result in first row
user_data["age_on_platform"].head(1)

0   151 days
Name: age_on_platform, dtype: timedelta64[ns]

# Labelling the Raw data

Now comes the code that will based on the rules mentioned below label the provided data, so it can be used as trainning data for the classifer.

This tables defines the set of rules used to assign labels for Traning data

| label               | age_on_platform      | last_activity             | num_modules_comsumed        | num_of_days_food_tracked | num_glucose_tracked         | watching_videos  |
|---------------------|----------------------|---------------------------|-----------------------------|--------------------------|-----------------------------|------------------|
| Generic (ignore)    | Converted to days    | to be Measured from 16Apr | Good >= 3/week Bad < 3/week | Good >= 30 Bad < 30      | Good >= 4/week Bad < 4/week | Good = 1 Bad = 0 |
| good_new_user       | >= 30 days && < 180  | <= 2 days                 | >= 12                       | >= 20                    | >= 16                       | Good = 1         |
| bad_new_user        | >= 30 days && < 180  | > 2 days                  | < 12                        | < 20                     | < 16                        | Bad = 0          |
| good_mid_term_user  | >= 180 days && < 360 | <= 7 days                 | >= 48                       | >= 30                    | >= 96                       | Good = 1         |
| bad_mid_term_user   | >= 180 days && <360  | > 7 days                  | < 48                        | < 30                     | < 96                        | Bad = 0          |
| good_long_term_user | >= 360 days          | <= 14 days                | >= 48                       | >= 30                    | >= 192                      | Good = 1         |
| bad_long_term_user  | >= 360 days          | > 14 days                 | < 48                        | < 30                     | < 192                       | Bad = 0          |

In [55]:
one_month = 30
#one_month = one_month.astype(int)
six_month = 180

## There are empty rows in *last_activity* and *first_login* !! Correct that first 

In [56]:
for index, row in user_data.iterrows():
    if (row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int) >= one_month and (row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int) < six_month:
        row["label"] = 1
    elif ((row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int)) >=180  and ((row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int))< 360:
        row["label"] = 3
    elif ((row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int))>= 360:
        row["label"] = 5
        
    

AttributeError: 'float' object has no attribute 'astype'

In [57]:
for index, row in user_data.iterrows():
    if row["age_on_platform"] >= np.timedelta64(30, 'D') and row["age_on_platform"] < np.timedelta64(180, 'D'):
        row["label"] = 1
    elif ((row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int)) >=180  and ((row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int))< 360:
        row["label"] = 3
    elif ((row["age_on_platform"] / np.timedelta64(1, 'D')).astype(int))>= 360:
        row["label"] = 5
        
    

TypeError: Cannot compare type 'NaTType' with type 'timedelta64'

In [53]:
(user_data["age_on_platform"].head(1) / np.timedelta64(1, 'D')).astype(int)

0    151
Name: age_on_platform, dtype: int64

In [60]:
np.timedelta64('180', 'D')

numpy.timedelta64(180,'D')